# 📄 Módulo 2: Análisis de Texto con Amazon Comprehend

**Duración**: 15 minutos  
**Objetivo**: Aprender a extraer información de textos científicos usando IA

## 🎯 ¿Qué aprenderás?
- Analizar texto científico para extraer entidades y conceptos clave
- Detectar sentimientos en reportes y alertas
- Identificar frases importantes automáticamente
- Aplicar IA a documentación técnica real

## 🔧 Configuración Inicial

In [ ]:
# Instalar dependencias
import sys
!{sys.executable} -m pip install boto3 matplotlib pandas seaborn --quiet

print("✅ Dependencias instaladas")

In [ ]:
# Importar librerías
import boto3
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import ipywidgets as widgets
from IPython.display import display

# Configurar estilo
plt.style.use('default')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")

## 🔑 Configurar Credenciales AWS

In [ ]:
# Configurar cliente Comprehend
try:
    comprehend = boto3.client('comprehend', region_name='us-east-1')
    # Probar conexión
    comprehend.detect_dominant_language(Text="Test connection")
    print("✅ Credenciales AWS configuradas correctamente")
except Exception as e:
    print("❌ Error con credenciales. Configurar manualmente en la siguiente celda")
    print(f"Error: {e}")

In [ ]:
# Configuración manual (si es necesario)
AWS_ACCESS_KEY_ID = "TU_ACCESS_KEY_AQUI"
AWS_SECRET_ACCESS_KEY = "TU_SECRET_KEY_AQUI"

if AWS_ACCESS_KEY_ID != "TU_ACCESS_KEY_AQUI":
    comprehend = boto3.client(
        'comprehend',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name='us-east-1'
    )
    print("✅ Credenciales configuradas manualmente")
else:
    print("⚠️ Reemplaza las credenciales si es necesario")

## 📊 Datos Científicos de Muestra

In [ ]:
# Textos científicos
SCIENTIFIC_TEXTS = {
    "Reporte Sísmico": """
    El Instituto Geofísico del Perú reporta actividad sísmica significativa en la región sur. 
    Los análisis preliminares indican un evento de magnitud 6.2 Mw con epicentro a 45 kilómetros 
    de profundidad. Las estaciones de monitoreo registraron ondas P y S características de 
    sismos tectónicos. Se recomienda mantener protocolos de seguridad en las zonas urbanas cercanas.
    """,
    
    "Alerta de Emergencia": """
    ALERTA SÍSMICA: Se ha detectado un sismo de magnitud 5.8 en la costa peruana. 
    Tiempo estimado de llegada a Lima: 2 minutos. Busque refugio inmediatamente. 
    Aléjese de ventanas y objetos que puedan caer. Mantenga la calma y siga los procedimientos.
    """,
    
    "Abstract Científico": """
    This study analyzes volcanic activity patterns in the Andes mountain range using 
    satellite imagery and seismic data. Results show increased thermal anomalies 
    correlating with micro-seismic events. The findings suggest enhanced monitoring 
    protocols for early volcanic eruption detection.
    """
}

print("📚 Textos científicos cargados:")
for i, (title, text) in enumerate(SCIENTIFIC_TEXTS.items(), 1):
    word_count = len(text.split())
    print(f"  {i}. {title} ({word_count} palabras)")

## 🔍 Función de Análisis

In [ ]:
def analyze_text_with_comprehend(text, text_name):
    """Analiza texto usando Amazon Comprehend"""
    try:
        print(f"🔄 Analizando: {text_name}")
        
        # Limpiar texto
        clean_text = text.strip().replace('\n', ' ')
        
        # Detectar idioma
        language_result = comprehend.detect_dominant_language(Text=clean_text)
        dominant_language = language_result['Languages'][0]['LanguageCode']
        print(f"  📝 Idioma: {dominant_language}")
        
        results = {
            'text_name': text_name,
            'language': dominant_language,
            'word_count': len(clean_text.split())
        }
        
        # Detectar entidades
        entities_result = comprehend.detect_entities(
            Text=clean_text,
            LanguageCode=dominant_language
        )
        results['entities'] = entities_result['Entities']
        print(f"  🏷️ Entidades: {len(results['entities'])}")
        
        # Frases clave
        keyphrases_result = comprehend.detect_key_phrases(
            Text=clean_text,
            LanguageCode=dominant_language
        )
        results['key_phrases'] = keyphrases_result['KeyPhrases']
        print(f"  🔑 Frases clave: {len(results['key_phrases'])}")
        
        # Sentimientos
        sentiment_result = comprehend.detect_sentiment(
            Text=clean_text,
            LanguageCode=dominant_language
        )
        results['sentiment'] = sentiment_result['Sentiment']
        results['sentiment_scores'] = sentiment_result['SentimentScore']
        print(f"  😊 Sentimiento: {results['sentiment']}")
        
        results['success'] = True
        return results
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return {'success': False, 'error': str(e)}

print("✅ Función de análisis creada")

## 📊 Función de Visualización

In [ ]:
def visualize_results(analysis_result):
    """Visualiza resultados del análisis"""
    if not analysis_result.get('success'):
        print(f"❌ Error: {analysis_result.get('error')}")
        return
    
    text_name = analysis_result['text_name']
    
    # Crear gráficos
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Análisis: {text_name}', fontsize=16, fontweight='bold')
    
    # 1. Tipos de entidades
    if analysis_result['entities']:
        entity_types = [e['Type'] for e in analysis_result['entities']]
        entity_counts = Counter(entity_types)
        
        ax1.bar(entity_counts.keys(), entity_counts.values(), color='lightblue')
        ax1.set_title('Entidades Detectadas')
        ax1.tick_params(axis='x', rotation=45)
    else:
        ax1.text(0.5, 0.5, 'Sin entidades', ha='center', transform=ax1.transAxes)
        ax1.set_title('Entidades Detectadas')
    
    # 2. Frases clave
    if analysis_result['key_phrases']:
        top_phrases = sorted(analysis_result['key_phrases'], 
                           key=lambda x: x['Score'], reverse=True)[:6]
        phrases = [p['Text'][:25] + '...' if len(p['Text']) > 25 else p['Text'] for p in top_phrases]
        scores = [p['Score'] for p in top_phrases]
        
        ax2.barh(phrases, scores, color='lightgreen')
        ax2.set_title('Frases Clave (Top 6)')
        ax2.set_xlabel('Relevancia')
    else:
        ax2.text(0.5, 0.5, 'Sin frases clave', ha='center', transform=ax2.transAxes)
        ax2.set_title('Frases Clave')
    
    # 3. Sentimientos
    sentiment_scores = analysis_result['sentiment_scores']
    sentiments = ['Positive', 'Negative', 'Neutral', 'Mixed']
    scores = [sentiment_scores[s] for s in sentiments]
    colors = ['green', 'red', 'gray', 'orange']
    
    ax3.pie(scores, labels=sentiments, colors=colors, autopct='%1.1f%%')
    ax3.set_title(f'Sentimientos\n(Dominante: {analysis_result["sentiment"]})')
    
    # 4. Estadísticas
    ax4.axis('off')
    stats = f"""📊 Estadísticas:

Palabras: {analysis_result['word_count']}
Entidades: {len(analysis_result['entities'])}
Frases clave: {len(analysis_result['key_phrases'])}
Idioma: {analysis_result['language'].upper()}"""
    ax4.text(0.1, 0.7, stats, fontsize=12, verticalalignment='top')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar entidades
    if analysis_result['entities']:
        print(f"\n🏷️ ENTIDADES EN '{text_name.upper()}'")
        print("=" * 50)
        for entity in analysis_result['entities'][:8]:
            print(f"• {entity['Text']} ({entity['Type']}) - {entity['Score']:.2f}")

print("✅ Función de visualización creada")

## 🚀 Ejercicio 1: Análisis de Reporte Sísmico

In [ ]:
# Analizar reporte sísmico
text_name = "Reporte Sísmico"
text_content = SCIENTIFIC_TEXTS[text_name]

print(f"📄 Analizando: {text_name}")
print("-" * 50)

# Realizar análisis
seismic_analysis = analyze_text_with_comprehend(text_content, text_name)

# Visualizar
if seismic_analysis.get('success'):
    visualize_results(seismic_analysis)

## 🚨 Ejercicio 2: Análisis de Alerta

In [ ]:
# Analizar alerta
text_name = "Alerta de Emergencia"
text_content = SCIENTIFIC_TEXTS[text_name]

print(f"🚨 Analizando: {text_name}")
print("-" * 50)

alert_analysis = analyze_text_with_comprehend(text_content, text_name)

if alert_analysis.get('success'):
    visualize_results(alert_analysis)

## 🎛️ Widget Interactivo

In [ ]:
# Widget para análisis interactivo
def interactive_analysis(text_selection):
    text_content = SCIENTIFIC_TEXTS[text_selection]
    print(f"🔍 Analizando: {text_selection}")
    print("=" * 50)
    
    result = analyze_text_with_comprehend(text_content, text_selection)
    if result.get('success'):
        visualize_results(result)
    return result

# Crear widget
text_selector = widgets.Dropdown(
    options=list(SCIENTIFIC_TEXTS.keys()),
    description='Texto:'
)

interactive_widget = widgets.interactive(interactive_analysis, text_selection=text_selector)
display(interactive_widget)

## 🎓 Casos de Uso Reales

### Amazon Comprehend en Ciencias:
- 📊 **Análisis de literatura**: Procesar miles de papers automáticamente
- 🚨 **Sistemas de alerta**: Analizar tono y urgencia de comunicaciones
- 🔍 **Extracción de datos**: Identificar entidades científicas en textos
- 📈 **Análisis de tendencias**: Detectar patrones en grandes volúmenes de texto
- 📝 **Clasificación**: Organizar documentos por tema y sentimiento

## ✅ Validación del Módulo

In [ ]:
def validate_module():
    checks = {
        "Credenciales configuradas": False,
        "Análisis realizado": False,
        "Widget usado": False
    }
    
    try:
        comprehend.detect_dominant_language(Text="Test")
        checks["Credenciales configuradas"] = True
    except:
        pass
    
    if 'seismic_analysis' in globals() and seismic_analysis.get('success'):
        checks["Análisis realizado"] = True
        checks["Widget usado"] = True
    
    print("📋 VALIDACIÓN MÓDULO 2 - COMPREHEND")
    print("=" * 40)
    
    for check, status in checks.items():
        icon = "✅" if status else "❌"
        print(f"{icon} {check}")
    
    completed = sum(checks.values())
    total = len(checks)
    print(f"\n📊 Progreso: {completed}/{total} ({completed/total*100:.0f}%)")
    
    if completed >= 2:
        print("\n🎉 ¡MÓDULO COMPLETADO!")
        print("➡️ Continúa con Módulo 3: Textract")
    else:
        print("\n⚠️ Completa los ejercicios faltantes")
    
    return completed >= 2

module_completed = validate_module()

---

## 🚀 Próximo Módulo

**📄 Módulo 3: OCR con Amazon Textract**

---

*Módulo 2 de 6 completado* ✅